In [1]:
#loading in the data
import pandas as pd; import numpy as np

In [2]:
#loading play by play data and
play_by_play = pd.read_csv("/Users/meganrobertson/Desktop/Basketball_Data/Play_by_Play_New.csv")
play_by_play = play_by_play.loc[play_by_play['Season'] == 2016]

In [3]:
#extracting when substitutions are made
subs = play_by_play.loc[(play_by_play['General_Description'] == 'Substitution') | (play_by_play['Play_Clock_Time'] == 720)]

In [4]:
#adding start/end time to the sub data
subs = subs.sort_values(['Game_id', 'Period', 'Play_Clock_Time'], ascending =[True, True, False]) #sorting 
subs = subs.reset_index()
subs['start'] = subs['Play_Clock_Time']
subs['end'] = 0
subs = subs.reset_index()

for i in range(0, subs.shape[0]-1):
    if subs.iloc[i]['Game_id'] == subs.iloc[i+1]['Game_id']:
        subs.loc[i, 'end'] = subs.iloc[i+1]['Play_Clock_Time'] - 1
    else:
        pass

In [5]:
#reading in the possession log and formatting the time
possess = pd.read_table('/Users/meganrobertson/Desktop/Basketball_Data/2016-17_nba_possession_log.txt')
possess['GAME_CLOCK_START'] = (possess['GAME_CLOCK_START']/10).astype(int)
possess['GAME_CLOCK_END'] = (possess['GAME_CLOCK_END']/10).astype(int)

In [6]:
#creating player information columns
possess['Player1'] = 'NA'
possess['Player2'] = 'NA'
possess['Player3'] = 'NA'
possess['Player4'] = 'NA'
possess['Player5'] = 'NA'
possess['Player6'] = 'NA'
possess['Player7'] = 'NA'
possess['Player8'] = 'NA'
possess['Player9'] = 'NA'
possess['Player10'] = 'NA'

In [8]:
#looping through each possession
for i in range(0, possess.shape[0]): 
    
    #which substitutions match the possession
    players = subs.loc[(subs['Game_id'] == possess.iloc[i]['GAME_ID']) & (subs['Period'] == possess.iloc[i]['PERIOD']) & (subs['start'] > possess.iloc[i]['GAME_CLOCK_START'] + 4) & (subs['end'] < possess.iloc[i]['GAME_CLOCK_START']) - 4, ['Player1', 'Player2', 'Player3', 'Player4', 'Player5', 'Player6', 'Player7', 'Player8', 'Player9', 'Player10']]
    if (i % 1000) == 0: #printing every 1000 rows through the loop
        print(i)
    try: #some possessions didn't have matching subs, so catching that error
        possess.loc[i, ['Player1', 'Player2', 'Player3', 'Player4', 'Player5', 'Player6', 'Player7', 'Player8', 'Player9', 'Player10']] = players.iloc[0]
    except: #if no players, "NA" for each player won't be overwritten
        pass

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [9]:
#saving the possession with player info
possess.to_csv('mod_all_players.csv')

In [ ]:
#reading in all star and player map data, editing strings for matching
all_stars = pd.read_csv('/Users/meganrobertson/Desktop/Nothing_But_Elastic_Nets/all_star_info.csv')
all_stars['PLAYERS'] = all_stars['PLAYERS'].str.lower()
player_map = pd.ExcelFile('/Users/meganrobertson/Desktop/Basketball_Data/Player_Map.xlsx').parse()
player_map['PLAYERS'] = player_map['Name'].str.lower()

#combine all_stars with player_map
player_map = player_map.merge(all_stars, how ="left", on ="PLAYERS")

#function to find how many all_stars are in each possession
def count_all_stars(row):
    names = player_map.loc[(player_map['Player_id'].isin(row[['Player1', 'Player2', 'Player3', 'Player4', 'Player5', 'Player6', 'Player7', 'Player8', 'Player9', 'Player10']])) & (player_map['ALL-STAR GAMES'] > 0)]
    row['num_all_stars'] = names.shape[0]
    return(row)

#applying the counting function to the data
data = possess.apply(count_all_stars, axis=1)
data.to_csv('/Users/meganrobertson/Desktop/Nothing_But_Elastic_Nets/data_all_stars.csv')